In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner
import hazm
from collections import defaultdict




In [2]:
data = pd.read_csv('./culture.csv')

In [3]:
def delete_stop_words(word):    
    if len(word)<3:
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

In [4]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

In [5]:
text = data['mText'][0]
all_words = nltk.word_tokenize(unicode(text,'utf-8'))
words = clean_text(text)

In [6]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [8]:
position_dict= defaultdict(list)
for i in words:
    position_dict[i['Word']].append(i['Position'])
    
for i in position_dict:
    position_dict[i] = sorted(position_dict[i])

In [10]:
life_time = defaultdict(int)
for i in position_dict:
    n = position_dict[i][-1]-position_dict[i][0] -1 
    if n == -1 :
        n = 0
    life_time[i] = float(n)/len(all_words)

In [11]:
words_dict = []
for i in position_dict:
    n = defaultdict()
    n['Word'] = i
    n['Position'] = position_dict[i]
    n['life_time'] = life_time[i]
    n['tf'] = len(position_dict[i])
    words_dict.append(n)

In [12]:
mean = float(sum([sum(position_dict[i]) for i in position_dict]))/len(position_dict)
for word in words_dict:
    s = 0 
    for position in word['Position']:
        s += (position-mean)**2
    word['Var'] = s/len(words_dict)

In [17]:
for word in words_dict:
    word['Weight'] = (word['life_time'] + word['tf']) - word['Var']

    

In [21]:
words_dict

[defaultdict(None,
             {'Position': [88],
              'Var': 8.80864918209013,
              'Weight': -7.8086491820901305,
              'Word': u'\u0645\u062f\u0639\u0648',
              'life_time': 0.0,
              'tf': 1}),
 defaultdict(None,
             {'Position': [61],
              'Var': 0.23917860966336396,
              'Weight': 0.7608213903366361,
              'Word': u'\u0628\u0646\u0627',
              'life_time': 0.0,
              'tf': 1}),
 defaultdict(None,
             {'Position': [1, 64],
              'Var': 66.78069530048771,
              'Weight': -64.19578964011035,
              'Word': u'\u06af\u0632\u0627\u0631\u0634',
              'life_time': 0.5849056603773585,
              'tf': 2}),
 defaultdict(None,
             {'Position': [105],
              'Var': 26.466536846696417,
              'Weight': -25.466536846696417,
              'Word': u'\u067e\u06cc\u0627\u0645',
              'life_time': 0.0,
              'tf': 1}),
 defa